# **Laboratorio 1**

**Integrantes**


**Estudiante 1:** 
Sebastian Contreras Salazar - 202020903

**Estudiante 2:**
Juan Miguel Manrique - 202021447

**Estudiante 3:**
Andres

# **Carga de datos**

Como primer paso de este laboratorio, se requiere cargar los datos:

In [1]:
# Imports
!pip install matplotlib seaborn scikit-learn scipy pandas numpy joblib
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from joblib import dump, load

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import scipy.stats as stats


^C


Ahora cargamos los datos correspondientes:

In [45]:
#Carga de de datos
datos_entrenamiento = pd.read_csv('data/datos_entrenamiento_laboratorio1(train_data).csv')
datos_validacion = pd.read_csv('data/datos_validacion_laboratorio1(test_data).csv')

Revisamos las dimensiones:

In [46]:
#Dimensiones
datos_entrenamiento.shape

(57934, 50)

Miramos los primeros datos:

In [47]:
#Mostramos los primeros 5 registros del dataset
print("Primeras filas: ")
display(datos_entrenamiento.head())

Primeras filas: 


,Jugador,Nacionalidad,Posicion,Edad,Dia_partido,Goles,Tiros Totales,xG,npxG,xAG,...,Malos_controles,Perdida_balon,Pases_recibidos,Pases_progresivos_recibidos,Faltas_cometidas,Centros,Duelos_aereos_ganados,%_de_duelos_aereos_ganados,market_value,contract_date
0,Bruno Fernandes,pt POR,"FW,AM",29-343,"Friday August 16, 2024",0,6,1.2,1.2,0.0,...,1,2,51,7,0,8,0,0.0,€50.00m,30.06.2027
1,Marcus Rashford,eng ENG,LW,26-290,"Friday August 16, 2024",0,0,0.0,0.0,0.7,...,2,1,37,9,1,1,1,100.0,€50.00m,30.06.2026
2,Amad Diallo,ci CIV,RW,22-036,"Friday August 16, 2024",0,0,0.0,0.0,0.1,...,2,2,26,7,0,2,0,0.0,-,30.06.2027
3,Amad Diallo,ci CIV,RW,22-036,"Friday August 16, 2024",-5,0,0.0,0.0,0.1,...,2,2,26,7,0,2,0,0.0,€1000.00m,30.06.2030
4,Alejandro Garnacho,ar ARG,RW,20-046,"Friday August 16, 2024",0,2,0.7,0.7,0.2,...,2,1,12,3,0,1,1,50.0,€45.00m,30.06.2028


Con esto en cuenta podemos proceder a entender y visualizar los datos.

## **Entendimiento y visualización de los Datos**

1. Explorar los datos:

In [48]:
# Información general y tipos de datos
datos_entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57934 entries, 0 to 57933
Data columns (total 50 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Jugador                         57934 non-null  object 
 1   Nacionalidad                    57902 non-null  object 
 2   Posicion                        57934 non-null  object 
 3   Edad                            57886 non-null  object 
 4   Dia_partido                     57934 non-null  object 
 5   Goles                           57934 non-null  int64  
 6   Tiros Totales                   57934 non-null  int64  
 7   xG                              57934 non-null  float64
 8   npxG                            57934 non-null  float64
 9   xAG                             57934 non-null  float64
 10  xAG.1                           57934 non-null  float64
 11  Acciones_que_crean_tiros        57934 non-null  int64  
 12  Pases_intentados                

Con esto, tambien podemos mirar las estadísticas descriptivas:

In [49]:

# Estadísticas descriptivas
display(datos_entrenamiento.describe(include='all'))


,Jugador,Nacionalidad,Posicion,Edad,Dia_partido,Goles,Tiros Totales,xG,npxG,xAG,...,Malos_controles,Perdida_balon,Pases_recibidos,Pases_progresivos_recibidos,Faltas_cometidas,Centros,Duelos_aereos_ganados,%_de_duelos_aereos_ganados,market_value,contract_date
count,57934,57902,57934,57886,57934,57934.000000,57934.000000,57934.000000,57934.000000,57934.000000,...,57934.000000,57934.000000,57934.000000,57934.000000,57934.000000,57934.000000,57934.000000,36977.000000,45040,45018
unique,2242,110,452,7114,198,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95,11
top,Vitinha,fr FRA,CB,24-231,"Saturday May 10, 2025",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€5.00m,30.06.2028
freq,166,9351,9537,37,997,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2332,11316
mean,NaN,NaN,NaN,NaN,NaN,0.513015,0.792402,0.088635,0.080473,0.062784,...,0.950133,0.574033,25.518383,2.382470,0.781337,1.133652,0.834795,48.814187,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,16.440600,1.158538,0.209810,0.186549,0.152033,...,1.221368,0.931852,19.001270,3.158174,1.005105,2.005869,1.305149,39.560765,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,-5.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,22.000000,1.000000,0.000000,0.000000,0.000000,50.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,0.100000,0.100000,0.100000,...,1.000000,1.000000,36.000000,4.000000,1.000000,1.000000,1.000000,100.000000,NaN,NaN


Con describe podemos ver las variables que existen y las primeras estadisticas para conocer como los datos estan presentes. Estas son algunos datos que podemos sacar a partir de lo obtenido:
## Análisis y conclusiones relevantes sobre los datos

- **Cantidad de registros:** El dataset cuenta con 47,451 registros, representando múltiples partidos y jugadores.
- **Jugadores únicos:** Hay 2,242 jugadores diferentes, lo que indica una buena diversidad de la muestra.
- **Nacionalidades:** Se identifican 110 nacionalidades distintas, mostrando diversidad internacional.
- **Posiciones:** Existen 452 valores únicos en la columna de posición, lo que puede deberse a combinaciones de roles o errores de captura.
- **Edad:** La edad de los jugadores varía entre 16 y 42 años, con una media de 26 años, lo que es consistente con la edad típica de futbolistas profesionales.
- **Goles y tiros:** El promedio de goles por registro es bajo (0.6), lo que es esperable dado que la mayoría de partidos tienen pocos goles por jugador. El máximo de goles en un registro es 995, lo que sugiere posibles errores o registros acumulados.
- **Métricas avanzadas:** Las variables como xG, npxG, xA y PSxG tienen medias bajas, lo que es coherente con su interpretación como probabilidades o expectativas por partido.
- **Pases y acciones:** El número de pases intentados y completados varía ampliamente, con valores máximos altos que pueden indicar partidos con mucha participación o errores de captura.
- **Porcentajes:** Los porcentajes de regates exitosos y duelos aéreos ganados tienen valores entre 0 y 100, aunque hay que revisar los valores negativos o superiores a 100 que podrían indicar errores.
- **Valores nulos:** Algunas columnas presentan valores nulos, especialmente en métricas avanzadas y porcentajes, lo que debe ser tratado antes del modelado.
- **Valor de mercado:** El valor de mercado (`market_value`) tiene una media de aproximadamente 31 millones de euros, pero con una desviación estándar muy alta y valores máximos que llegan hasta casi 10 mil millones, lo que sugiere la presencia de outliers o errores de captura.
- **Fechas:** Hay 198 fechas de partido únicas y 11 fechas de contrato diferentes, lo que puede ayudar a analizar la evolución temporal de los jugadores.

### Datos a tener en cuenta:

- El dataset es amplio y diverso, lo que es positivo para el modelado.
- Existen posibles errores o outliers en variables como goles, valor de mercado y porcentajes, que deben ser revisados y tratados.
- Hay columnas con valores nulos que requieren imputación o eliminación según su importancia.
- La variedad de posiciones y nacionalidades permitirá analizar el impacto de estas variables en el valor de mercado.
- Es recomendable realizar una limpieza adicional para asegurar la calidad de los datos antes de construir el modelo de regresión.

Teniendo esto en cuenta, también podemos mirar las columnas que tenemos presentes en el dataset:

In [50]:
datos_entrenamiento.columns

Index(['Jugador', 'Nacionalidad', 'Posicion', 'Edad', 'Dia_partido', 'Goles',
       'Tiros Totales', 'xG', 'npxG', 'xAG', 'xAG.1',
       'Acciones_que_crean_tiros', 'Pases_intentados', 'Pases_intentados.1',
       'Pases_intentados.2', 'Pases_progresivos', 'Pases_progresivos.1',
       'Regates_exitosos', 'Regates_exitosos.1', 'Pases_medios_completados',
       'Pases_largos_completados', 'xAG.2', 'xAG.3', 'xA',
       'Pases_en_ultimo_tercio', 'Pases_balon_vivo', 'Pases_balon_muerto',
       'Pases_al_hueco', 'Pases_centros', 'Pases_completados',
       'Pases_completados.1', 'Pases_completados.2', 'Pases_fuera_de_juego',
       'Valla_no_vencida', 'Errores_defnsivos_ocasion_tiro',
       'Toques_en_zona_ofensiva', 'Toques_en_area_rival',
       '%_de_regates_exitosos', 'Veces_que_fue_barrido_regate',
       'Acarreos_ultimo_tercio', 'Malos_controles', 'Perdida_balon',
       'Pases_recibidos', 'Pases_progresivos_recibidos', 'Faltas_cometidas',
       'Centros', 'Duelos_aereos_ganad

Y ahora podemos sacar un diccionario por cada columna, que no se repiten, respectivamente:

| Columna                         | Tipo         | Descripción                                                                 |
|----------------------------------|--------------|-----------------------------------------------------------------------------|
| Jugador                         | Texto        | Nombre del jugador de fútbol.                                               |
| Nacionalidad                    | Texto        | País de origen del jugador.                                                 |
| Posicion                        | Texto        | Posición en el campo (ej. Defensa, Delantero, Portero).                     |
| Edad                            | Entero       | Edad del jugador en años.                                                   |
| Dia_partido                     | Fecha/Text   | Día en que se disputó el partido.                                           |
| Goles                           | Entero       | Número de goles anotados en el partido.                                     |
| Tiros Totales                   | Entero       | Total de tiros intentados (a puerta + fuera).                               |
| xG                              | Float        | Goles esperados (Expected Goals).                                           |
| npxG                            | Float        | Goles esperados sin penales.                                                |
| xAG                             | Float        | Asistencias de goles esperadas (Expected Assisted Goals).                   |
| Acciones_que_crean_tiros        | Entero       | Acciones del jugador que generaron un tiro.                                 |
| Pases_intentados                | Entero       | Número total de pases intentados.                                           |
| Pases_progresivos               | Entero       | Pases que avanzaron significativamente el balón hacia la portería rival.    |
| Regates_exitosos                | Entero       | Número de regates completados con éxito.                                    |
| Pases_medios_completados        | Entero       | Pases completados a distancia media (10–30m).                               |
| Pases_largos_completados        | Entero       | Pases largos completados (>30m).                                            |
| xA                              | Float        | Asistencias esperadas (Expected Assists).                                   |
| Pases_en_ultimo_tercio          | Entero       | Pases completados en el último tercio del campo.                            |
| Pases_balon_vivo                | Entero       | Pases realizados con balón en movimiento.                                   |
| Pases_balon_muerto              | Entero       | Pases realizados en jugadas a balón parado (ej. córners, tiros libres).     |
| Pases_al_hueco                  | Entero       | Pases filtrados entre defensores.                                           |
| Pases_centros                   | Entero       | Centros realizados al área rival.                                           |
| Pases_completados               | Entero       | Total de pases completados.                                                 |
| Pases_fuera_de_juego            | Entero       | Pases que resultaron en fuera de juego.                                     |
| Valla_no_vencida                | Entero       | Número de veces que el portero mantuvo la portería en cero.                 |
| Errores_defnsivos_ocasion_tiro  | Entero       | Errores defensivos que ocasionaron un tiro rival.                           |
| Toques_en_zona_ofensiva         | Entero       | Número de toques del jugador en campo rival.                                |
| Toques_en_area_rival            | Entero       | Toques del jugador dentro del área rival.                                   |
| %_de_regates_exitosos           | Float (%)    | Porcentaje de regates completados con éxito.                                |
| Veces_que_fue_barrido_regate    | Entero       | Veces que el jugador fue barrido mientras intentaba un regate.              |
| Acarreos_ultimo_tercio          | Entero       | Conducciones de balón que llegaron al último tercio del campo.              |
| Malos_controles                 | Entero       | Veces que el jugador perdió control del balón.                              |
| Perdida_balon                   | Entero       | Veces que el jugador perdió posesión.                                       |
| Pases_recibidos                 | Entero       | Número de pases recibidos por el jugador.                                   |
| Pases_progresivos_recibidos     | Entero       | Pases recibidos que avanzaron significativamente hacia la portería rival.   |
| Faltas_cometidas                | Entero       | Faltas cometidas por el jugador.                                            |
| Centros                         | Entero       | Centros realizados al área rival.                                           |
| Duelos_aereos_ganados           | Entero       | Duelos aéreos que el jugador ganó.                                          |
| %_de_duelos_aereos_ganados      | Float (%)    | Porcentaje de duelos aéreos ganados.                                        |
| market_value                    | Float (€)    | Valor estimado de mercado del jugador en euros.                             |
| contract_date                   | Fecha        | Fecha de finalización del contrato del jugador.                             |

Con esto, es posible ya tener un entendimiento claro de los datos, y se puede proceder a realizar el análisis de la calidad de los datos.

### Completitud

Vamos a comenzar con la completitud de lso datos, los cuales revisaremos las dimensiones de la calidad de los datos, revisando la existencia o no de un dato según un criterio. Para estos casos, revisaremos los datos si son nulos/faltantes.

### Validez

Comenzamos con la validez de los datos. En este punto vamos a revisar problemas de condificacion y transformacion en los datos que permitan determinar el valor de un dato faltante si es posible. De lo contrario se eliminan. 

In [51]:
#Revisamos las dimensiones
datos_entrenamiento.shape

(57934, 50)

Revisamos que datos existen que sean nulos:

In [53]:
(datos_entrenamiento.isnull().sum() / datos_entrenamiento.shape[0]).sort_values(ascending=False)

%_de_regates_exitosos             0.495771
%_de_duelos_aereos_ganados        0.361739
contract_date                     0.222943
market_value                      0.222564
Edad                              0.000829
Nacionalidad                      0.000552
Jugador                           0.000000
Posicion                          0.000000
Dia_partido                       0.000000
Goles                             0.000000
xAG.1                             0.000000
Acciones_que_crean_tiros          0.000000
Pases_intentados                  0.000000
Pases_intentados.1                0.000000
Tiros Totales                     0.000000
xG                                0.000000
npxG                              0.000000
xAG                               0.000000
Regates_exitosos                  0.000000
Pases_progresivos.1               0.000000
Pases_progresivos                 0.000000
Pases_intentados.2                0.000000
Regates_exitosos.1                0.000000
Pases_medio

Con esta información podemos identificar los siguientes elementos: %_de_regates_exitosos             0.495771
%_de_duelos_aereos_ganados        0.361739
contract_date                     0.222943
market_value                      0.222564
Edad                              0.000829
Nacionalidad                      0.000552
Los cuales son el panorama de porcentaje de los nulos. Como podemos observar, los porcentajes de Edad y Nacionalidad son pequeños, pero son datos relevantes, por lo cual se van a eliminar esas filas. Por otro lado, para contract_date y market_value, dado que podrian afectar el resto de los datos, si los reemplazamos con datos incorrectos o nuevos, podrian afectar el modelo, por lo cual vamos a buscar datos que puedan reemplazarse en ellos, siendo estos, si se encuentra algo de estos jugadores, los valores máximos. Ahora bien, para las columnas con muchos nulos que equivale a los porcentajes de regates exitosos y de duelos aereos ganados, vamos a agrupar por jugador, donde calcularemos el promedio de ese jugador en esos porcentajes con los datos que si tiene para rellenar sus valores faltantes con ese promedio. Sin embargo, si no tiene ningun dato, se elimina. Con todo lo dicho previamente, primero vamos a realizar la agrupación y solución propuesta para columnas numericas criticas como los porcentajes y mercado de valores:

### Validez